<a href="https://colab.research.google.com/github/shivangini831-sys/FitPulse-Health-Anomaly-Detection-From-Fitness-Devices/blob/main/Copy_of_anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
import plotly.express as px


In [ ]:
heart = pd.read_csv('/content/drive/MyDrive/heart_rate.csv')
sleep = pd.read_csv('/content/drive/MyDrive/sleep.csv')


In [ ]:
print(heart.columns)
print(sleep.columns)



Index(['timestamp', 'heart_rate'], dtype='object')
Index(['date', 'sleep_hours'], dtype='object')


In [ ]:
heart['timestamp'] = pd.to_datetime(heart['timestamp'])

# Prophet requires: ds (datetime) and y (value)
heart_prophet = heart.rename(columns={
    'timestamp': 'ds',
    'heart_rate': 'y'
})


In [ ]:
sleep['date'] = pd.to_datetime(sleep['date'])

In [ ]:
model = Prophet()
model.fit(heart_prophet[['ds', 'y']])

forecast = model.predict(heart_prophet[['ds']])

heart_prophet['yhat'] = forecast['yhat']
heart_prophet['residual'] = heart_prophet['y'] - heart_prophet['yhat']


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


In [ ]:
residual_threshold = 2 * heart_prophet['residual'].std()

heart_prophet['residual_anomaly'] = (
    heart_prophet['residual'].abs() > residual_threshold
)


In [ ]:
heart_prophet['threshold_anomaly'] = (
    (heart_prophet['y'] < 40) | (heart_prophet['y'] > 120)
)


In [ ]:
heart_prophet['final_anomaly'] = (
    heart_prophet['residual_anomaly'] |
    heart_prophet['threshold_anomaly']
)

heart_prophet['anomaly_label'] = heart_prophet['final_anomaly'].map({
    True: 'Anomalous',
    False: 'Normal'
})


In [ ]:
fig = px.line(
    heart_prophet,
    x='ds',
    y='y',
    title='Heart Rate Time-Series with Anomalies'
)

fig.add_scatter(
    x=heart_prophet[heart_prophet['final_anomaly']]['ds'],
    y=heart_prophet[heart_prophet['final_anomaly']]['y'],
    mode='markers',
    name='Anomaly'
)

fig.show()


In [ ]:
sleep['sleep_anomaly'] = (
    (sleep['sleep_hours'] < 4) |
    (sleep['sleep_hours'] > 10)
)


In [ ]:
fig2 = px.line(
    sleep,
    x='date',
    y='sleep_hours',
    title='Sleep Pattern Anomalies'
)

fig2.add_scatter(
    x=sleep[sleep['sleep_anomaly']]['date'],
    y=sleep[sleep['sleep_anomaly']]['sleep_hours'],
    mode='markers',
    name='Anomaly'
)

fig2.show()
